## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-12-16-51-20 +0000,wapo,Army charges gynecologist with secretly record...,https://www.washingtonpost.com/nation/2025/12/...
1,2025-12-12-16-50-39 +0000,bbc,Germany accuses Russia of air traffic control ...,https://www.bbc.com/news/articles/cvgrrnylzzyo...
2,2025-12-12-16-50-02 +0000,startribune,Live: Gov. Walz announces fraud prevention eff...,https://www.startribune.com/live-gov-walz-to-a...
3,2025-12-12-16-49-15 +0000,startribune,Deputies investigating 5 a.m. school parking l...,https://www.startribune.com/deputies-investiga...
4,2025-12-12-16-49-00 +0000,wsj,Historic Flooding in Washington Displaces Thou...,https://www.wsj.com/us-news/climate-environmen...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2338/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
42,trump,42
139,new,18
313,oil,15
316,venezuela,11
72,ukraine,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
197,2025-12-12-02-38-00 +0000,wsj,President Trump signs an executive order that ...,https://www.wsj.com/politics/policy/trump-sign...,106
207,2025-12-12-01-46-00 +0000,wsj,The Indiana Senate has rejected a redistrictin...,https://www.wsj.com/politics/policy/indiana-re...,99
100,2025-12-12-12-00-00 +0000,nypost,Fed chief frontrunner Kevin Hassett will do wh...,https://nypost.com/2025/12/12/business/fed-chi...,86
32,2025-12-12-15-39-00 +0000,wsj,New Epstein Photos Released by Democrats Show ...,https://www.wsj.com/politics/democrats-release...,84
28,2025-12-12-15-51-53 +0000,nypost,"Woody Allen, Andrew Mountbatten-Windsor, Bill ...",https://nypost.com/2025/12/12/us-news/woody-al...,83


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
197,106,2025-12-12-02-38-00 +0000,wsj,President Trump signs an executive order that ...,https://www.wsj.com/politics/policy/trump-sign...
229,54,2025-12-12-00-02-02 +0000,bbc,Reddit launches High Court challenge to Austra...,https://www.bbc.com/news/articles/clykk2yrl9ko...
207,53,2025-12-12-01-46-00 +0000,wsj,The Indiana Senate has rejected a redistrictin...,https://www.wsj.com/politics/policy/indiana-re...
74,47,2025-12-12-13-37-51 +0000,nyt,Nobel Peace Prize Winner Machado Says U.S. Hel...,https://www.nytimes.com/2025/12/11/world/ameri...
234,45,2025-12-11-23-51-00 +0000,latimes,L.A. County sues oil companies over unplugged ...,https://www.latimes.com/environment/story/2025...
100,44,2025-12-12-12-00-00 +0000,nypost,Fed chief frontrunner Kevin Hassett will do wh...,https://nypost.com/2025/12/12/business/fed-chi...
115,33,2025-12-12-11-19-36 +0000,nypost,Kilmar Abrego Garcia freed from immigration de...,https://nypost.com/2025/12/12/us-news/kilmar-a...
96,32,2025-12-12-12-07-00 +0000,wsj,U.S. Treasury Yields Rise as Market Prices Lim...,https://www.wsj.com/finance/investing/jgbs-mix...
10,31,2025-12-12-16-41-57 +0000,nypost,"Dow tumbles after hitting record high, while B...",https://nypost.com/2025/12/12/business/dow-tum...
320,30,2025-12-11-19-38-28 +0000,cbc,National museum CEO leaves role after report a...,https://www.cbc.ca/news/politics/pier-21-ceo-d...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
